In [1]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
! wget --no-check-certificate https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW2/BEST2010.zip
! unzip BEST2010.zip

--2020-08-19 05:13:11--  https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW2/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip’

BEST2010.zip        100%[===================>]   7.08M  27.4MB/s    in 0.3s    

2020-08-19 05:13:11 (27.4 MB/s) - ‘BEST2010.zip’ saved [7423530/7423530]

Archive:  BEST2010.zip
   creating: BEST2010/
  inflating: BEST2010/article.txt    
  inflating: BEST2010/encyclopedia.txt  
  inflating: BEST2010/news.txt       


# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation

As a reminder,
### Don't forget to shut down your instance on Gcloud when you are not using it ###


In [ ]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [ ]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
vocabuary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        vocabuary.add(word)
        total_word_count+=1

In [ ]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabuary in BEST2010 news dataset :\t'+ str(len(vocabuary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabuary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [ ]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

# We will use 1/total word count as a default value for unknown word
unk_value = math.pow(len(vocabuary),-1)

# Unigram

In this section, we will demonstrate how to build a unigram language model

In [ ]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    total_word_count =0
    for sentence in data:
        sentence +=  u'|</s>'
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            total_word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(total_word_count)
    return model

In [ ]:
model = getUnigramModel(train)

In [ ]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.476949890150093
Problability of a sentence 2.75827124812635e-16


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 513.97 and
452.66 on train and test data.

## TODO #1 Calculate perplexity

In [ ]:
def calculate_sentence_ln_prob(sentence, model):
    pass

def perplexity(test,model):
    pass

In [ ]:
print(perplexity(train,model))
print(perplexity(test,model))

513.9747870973313
452.65524668351003


## TODO #2 Please explain why this model give us such a high perplexity.

**Your answer**:  .........

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [ ]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.54 and 153.36 on train and test data.

## TODO #3 Write Bigram Model

In [ ]:
def getBigramModel(data):
    pass


model = getBigramModel(train)

## TODO #4 Write Perplexity for Bigram Model

Sum perplexity score at a sentence level, instead of word level

In [ ]:
def calculate_sentence_ln_prob(sentence, model):
    pass



def perplexity(test,model):
    pass

In [ ]:
print (perplexity(train,model) )
print (perplexity(test, model))

58.53517510765259
153.36386541850547


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 write Bigram with Laplace smoothing (Add-One Smoothing)

In [ ]:
#Laplace Smoothing
def getBigramWithLaplaceSmoothing(data):
    pass
model =getBigramWithLaplaceSmoothing(train)
print (perplexity(train,model) )
print (perplexity(test, model))

1231.113465432258
1390.82820339171


## TODO #6 Write Bigram with Interpolation
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [ ]:
#interpolation
def getBigramWithInterpolation(data):
    pass
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

39.899540475582384
107.26849422396057


the result perplexity on training and testing should be 

    1231.14, 1390.85 for Laplace smoothing
    39.90, 107.27 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language model from news and article domains to create a language model for the encyclopedia domain.

In [ ]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation on article data. The result perplexity should be around 466.17 

For your information, a bigram model with interpolation using ariticle data to test on encyclopedia data has a perplexity of 281.24

In [ ]:
#print perplexity of bigram with interpolation on article data        
print (perplexity(encyclo_data,model))

466.1682377516689


## TODO #7 
Write a model that produce 260.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

# Kneser-ney Smoothing



## TODO #8 (optional: Bonus 0.5 points) 
Implement kneser-ney smoothing with news and article data 

In [ ]:
print (perplexity(encyclo_data,model))